In [3]:
import os
from pathlib import Path

cwd = Path.cwd()
if cwd.name == "notebooks":
    os.chdir(cwd.parent)

In [2]:
import json

from sklearn.model_selection import train_test_split
from datasets import Dataset

In [4]:
dataset_path = "data/articles_expansion.json"

In [5]:
with open(dataset_path, 'r') as f:
    data = json.load(f)
    
## Extract the text from the articles and the questions
corpus = {} # (cid, text)
queries = {} # (qid, text)
relevant_docs = {} # (qid, set([relevant_cids]))

for article, article_data in data.items():
    cid = article
    corpus[cid] = article_data["texto"]
    questions = article_data["preguntas"]
    questions = list(questions.keys())

    for i, question in enumerate(questions):
        qid = f"{cid}_{i}"
        queries[qid] = question
        previous_qid = relevant_docs.get(qid, set())
        relevant_docs[qid] = previous_qid.union([cid])

# Split relevant_docs into train and eval datasets (80% train, 20% eval)
# Create HuggingFace datasets
qids = list(relevant_docs.keys())
train_qids, eval_qids = train_test_split(qids, test_size=0.2, random_state=42)

train_dict = {}
eval_dict = {}

train_dict["sentence_a"] = []
train_dict["sentence_b"] = []
train_dict["score"] = []

eval_dict["sentence_a"] = []
eval_dict["sentence_b"] = []
eval_dict["score"] = []

for qid in train_qids:
    for cid, text in corpus.items():
        train_dict["sentence_a"].append(queries[qid])
        train_dict["sentence_b"].append(text)
        train_dict["score"].append(1 if cid in relevant_docs[qid] else 0)

for qid in eval_qids:
    for cid, text in corpus.items():
        eval_dict["sentence_a"].append(queries[qid])
        eval_dict["sentence_b"].append(text)
        eval_dict["score"].append(1 if cid in relevant_docs[qid] else 0)

train_dataset = Dataset.from_dict(train_dict)
eval_dataset = Dataset.from_dict(eval_dict)

In [29]:
len(train_dataset) + len(eval_dataset)

427467

In [28]:
len(corpus)

267

In [19]:
for s_a, s_b in train_dataset[:5].items():
    print(s_a)
    print(s_b)
    # print(score)
    # print()
    

sentence_a
['¿Por qué es importante aprobar la programación de un semestre académico en la Universidad de Antioquia?', '¿Por qué es importante aprobar la programación de un semestre académico en la Universidad de Antioquia?', '¿Por qué es importante aprobar la programación de un semestre académico en la Universidad de Antioquia?', '¿Por qué es importante aprobar la programación de un semestre académico en la Universidad de Antioquia?', '¿Por qué es importante aprobar la programación de un semestre académico en la Universidad de Antioquia?']
sentence_b
['La Universidad de Antioquia como institución de servicio público, en cumplimiento de su función social, será siempre un centro de cultura y de ciencia que imparta a los estudiantes formación integral y los capacite para el ejercicio profesional en las diferentes áreas del quehacer humano.', 'La función esencial de la Universidad es la docente-investigativa en los campos de la ciencia, la técnica y la cultura y, ante ésta, las demás son 

In [24]:
for article, article_data in data.items():
    questions = article_data["preguntas"]
    print(f"Article: {article}, number of questions: {len(questions)}")

Article: ARTÍCULO 1., number of questions: 10
Article: ARTÍCULO 2., number of questions: 10
Article: ARTÍCULO 3., number of questions: 10
Article: ARTÍCULO 4., number of questions: 10
Article: ARTÍCULO 5., number of questions: 9
Article: ARTÍCULO 6., number of questions: 10
Article: ARTÍCULO 7., number of questions: 9
Article: ARTÍCULO 8., number of questions: 10
Article: ARTÍCULO 9., number of questions: 9
Article: ARTÍCULO 10., number of questions: 8
Article: ARTÍCULO 11., number of questions: 10
Article: ARTÍCULO 12., number of questions: 8
Article: ARTÍCULO 13., number of questions: 9
Article: ARTÍCULO 14., number of questions: 10
Article: ARTÍCULO 15., number of questions: 10
Article: ARTÍCULO 16., number of questions: 10
Article: ARTÍCULO 17., number of questions: 10
Article: ARTÍCULO 18., number of questions: 10
Article: ARTÍCULO 19., number of questions: 10
Article: ARTÍCULO 20., number of questions: 10
Article: ARTÍCULO 21., number of questions: 10
Article: ARTÍCULO 22., numbe

In [26]:
len(qids)

1601

In [22]:
relevant_docs

{'ARTÍCULO 1._0': {'ARTÍCULO 1.'},
 'ARTÍCULO 1._1': {'ARTÍCULO 1.'},
 'ARTÍCULO 1._2': {'ARTÍCULO 1.'},
 'ARTÍCULO 1._3': {'ARTÍCULO 1.'},
 'ARTÍCULO 1._4': {'ARTÍCULO 1.'},
 'ARTÍCULO 1._5': {'ARTÍCULO 1.'},
 'ARTÍCULO 1._6': {'ARTÍCULO 1.'},
 'ARTÍCULO 1._7': {'ARTÍCULO 1.'},
 'ARTÍCULO 1._8': {'ARTÍCULO 1.'},
 'ARTÍCULO 1._9': {'ARTÍCULO 1.'},
 'ARTÍCULO 2._0': {'ARTÍCULO 2.'},
 'ARTÍCULO 2._1': {'ARTÍCULO 2.'},
 'ARTÍCULO 2._2': {'ARTÍCULO 2.'},
 'ARTÍCULO 2._3': {'ARTÍCULO 2.'},
 'ARTÍCULO 2._4': {'ARTÍCULO 2.'},
 'ARTÍCULO 2._5': {'ARTÍCULO 2.'},
 'ARTÍCULO 2._6': {'ARTÍCULO 2.'},
 'ARTÍCULO 2._7': {'ARTÍCULO 2.'},
 'ARTÍCULO 2._8': {'ARTÍCULO 2.'},
 'ARTÍCULO 2._9': {'ARTÍCULO 2.'},
 'ARTÍCULO 3._0': {'ARTÍCULO 3.'},
 'ARTÍCULO 3._1': {'ARTÍCULO 3.'},
 'ARTÍCULO 3._2': {'ARTÍCULO 3.'},
 'ARTÍCULO 3._3': {'ARTÍCULO 3.'},
 'ARTÍCULO 3._4': {'ARTÍCULO 3.'},
 'ARTÍCULO 3._5': {'ARTÍCULO 3.'},
 'ARTÍCULO 3._6': {'ARTÍCULO 3.'},
 'ARTÍCULO 3._7': {'ARTÍCULO 3.'},
 'ARTÍCULO 3._8': {'

In [4]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("/nfs/home/marquez/personal/udea/models/matryoshka_paraphrase-multilingual-MiniLM-L12-v2/checkpoint-3300")
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

In [7]:
model.encode("Hello, my dog is cute").shape

(384,)